In [1]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Dropout, Flatten
import data

Using TensorFlow backend.


In [2]:
BATCH_SIZE = 128
EPOCHS = 12

In [3]:
(x_train, y_train), (x_test, y_test) = data.load_data()
input_shape = (data.IMAGE_HEIGHT, data.IMAGE_WIDTH, 1)

In [4]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(data.NUM_CLASSES, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer=keras.optimizers.Adadelta(), 
              metrics=['accuracy'])

In [5]:
class EarlyStopCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc') > 0.98):
            print("Reached 98% accuracy, cancel training")
            self.model.stop_training = True

In [6]:
callback = EarlyStopCallback()
model.fit(x_train, y_train, 
          batch_size=BATCH_SIZE, epochs=EPOCHS, 
          verbose=1, validation_data=(x_test, y_test),
          callbacks=[callback])

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 10s 171us/step - loss: 0.2597 - acc: 0.9199 - val_loss: 0.0541 - val_acc: 0.9832
Epoch 2/12
60000/60000 [==============================] - 7s 116us/step - loss: 0.0854 - acc: 0.9747 - val_loss: 0.0412 - val_acc: 0.9859
Epoch 3/12
60000/60000 [==============================] - 7s 115us/step - loss: 0.0642 - acc: 0.9807 - val_loss: 0.0349 - val_acc: 0.9885
Reached 98% accuracy, cancel training


In [7]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.03485422121305601
Test accuracy: 0.9885
